In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo-preview")


Let's create a vector Database

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb, os

embedding = OpenAIEmbeddings()

raw_documents = DirectoryLoader("./docs",glob="**/*.txt", show_progress=False).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(raw_documents)

chroma_directory = "chromadb"
if os.path.exists(chroma_directory):
    db = Chroma(embedding_function=embedding, persist_directory=chroma_directory)
else:
    db = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=chroma_directory)

In [4]:
from langchain.chains import RetrievalQA
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=db.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

Define a tool to fetch crime record

In [5]:
from langchain.agents import AgentExecutor, ZeroShotAgent
from ToolDefinitions import fetch_crime_records_tool, fetch_crime_record_stats_tool

tools = [fetch_crime_records_tool, fetch_crime_record_stats_tool]



In [6]:
from langchain.chains import LLMChain
prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix="You are a helpful assistant that helps people get crime records from offline database using the following tools. \n{instruction}\n\
            You have this chat history: {chat_history}\n\n",
    format_instructions="""use the following format:\n
    Question: the input question you must answer.
    Thought: you should always think about which tool to use.
    Action: the action to take, should always be only one of the tools '{tool_names}'
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: Summarize the observation which is the final answer to the original input question
    """,
    suffix="Begin:\nQuestion: {input}\nThought: {agent_scratchpad}",
    input_variables=['input', 'instruction'],
)
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [7]:
agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = [fetch_crime_records_tool],
    verbose=True
)


c:\Users\manoj\Documents\MANOJ\Github\Ask-API\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [8]:
from langchain.memory import ConversationTokenBufferMemory

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
    memory=ConversationTokenBufferMemory(memory_key="chat_history", max_token_limit=500,llm=llm),
    max_iterations=5,
    verbose=True
)

In [9]:
question = "Get crime records on crime type Homicide which happened near location 'Susan Spring'?"

In [10]:
instruction_from_llm = qa_chain.invoke({"query": question})['result']
agent_executor.run({"input": question,"instruction": instruction_from_llm})

c:\Users\manoj\Documents\MANOJ\Github\Ask-API\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that helps people get crime records from offline database using the following tools. 
I don't have the ability to access or retrieve live data, including current crime records, from external databases or APIs in real-time. To get crime records on the crime type Homicide near the location 'Susan Spring', you would need to use the specified API framework by setting the parameters for the location 'Susan Spring', the crime type 'Homicide', and relevant dates, then use the Search Endpoint to find matching crime records.
            You have this chat history: 



Get Crime record by location or crime type or date using /search_crime_records endpoint - tool/API: Fetches the crime records based on location or crime type or both using API endpoints,
    input should be a JSON string with location and crime_type as keys
Get crime database stats from /statistics endpoi

"A crime record of homicide was found near the location 'Susan Spring'. The record details are as follows: Crime Record ID: 4ad2c089-979c-406a-8ca3-17921d46a200, Location: 974 Susan Spring, Port Joshuaborough, VT 12878, Type of Crime: Homicide, Suspects: Alison Jackson, Victims: Linda Gentry, Witnesses: Anthony Dorsey, Description: Suggest bad number worker government style. Window institution attack future., Star election several four be western. Report industry order though fast., DateTime: 2024-02-03T09:39:31."

In [11]:
question = "Get statistics of the crime records database?"
instruction_from_llm = qa_chain.invoke({"query": question})['result']
agent_executor.run({"input": question,"instruction": instruction_from_llm})



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that helps people get crime records from offline database using the following tools. 
To get statistics of the crime records database, you would need to use the specific endpoints provided by the API framework that are designed for retrieving crime incidents, statistics, and related data. By leveraging these endpoints, users can access detailed information about crime trends, occurrences, and compile comprehensive statistics. This process involves identifying the relevant endpoints, utilizing the required parameters (like crime record ID for suspects, victims, and witnesses), and combining the retrieved information to present a complete statistical overview.
            You have this chat history: Human: I don't have the ability to access or retrieve live data, including current crime records, from external databases or APIs in real-time. To get crime records 

'The crime records database statistics are as follows: the oldest record date is 2024-01-01, and the newest record date is 2024-03-22. The value counts of different crime types are Homicide (214), Fraud (213), Burglary (202), Assault (191), and Robbery (180). There are 995 unique victims, 996 unique suspects, and 992 unique witnesses in the database.'